<a href="https://colab.research.google.com/github/juliosaopedro/Meus_Projetos/blob/master/Projeto_Enem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Projeto Previsão Nota ENEM </h1>

<center><img src=\"https://raw.githubusercontent.com/juliosaopedro/Meus_Projetos/blob/master/imagens/enem-logo.png\"></img></center>

Descubra as melhores notas de matemática do ENEM 2016
Você deverá criar um modelo para prever a nota da prova de matemática de quem participou do ENEM 2016.

Neste desafio você aprenderá:

<b> Lógica <br>
Análise de dados<br>
Estatística<br>
Regression<br>
Detalhes</b>

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). 
Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. <br>
Qualquer dúvida a respeito das colunas, consulte o Dicionário dos Microdados do Enem 2016.

Observações :<br>
O objetivo do desafio é avaliar a sua capacidade de resolver problemas, então ele pode ser resolvido em qualquer linguagem de programação 
ou ferramenta que você tenha facilidade. Mas recomendamos o uso de linguagens com maior ferramental de matemática e estatística como Python, R, Scala ou Julia.

Links : <br>
<a href="https://raw.githubusercontent.com/juliosaopedro/Meus_Projetos/master/datasets/train.csv">Dataset de treino</a> <br>
<a href="https://github.com/juliosaopedro/Meus_Projetos/blob/master/datasets/Dicionario_Microdados_Enem_2016.xlsx">Dicionario de Dados</a>


Importação de pacotes


In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

# nosso algoritmo
from xgboost import XGBRegressor 

# para nossas métricas
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

Importacao do arquivo para um DATAFRAME


In [0]:
df = pd.read_csv('train.csv')

Verificando os nossos dados

In [0]:
df.head()

Analisando os dados

In [0]:
# identificar o volume de dados do DataFrame
print("Variáveis: {}".format(df.shape[0]))

print("Entradas : {}".format(df.shape[1]))

# saltar uma linha
print("\n")


In [0]:
# verificar os tipos das entradas do dataset

display(df.dtypes)

Valores ausentes

In [0]:
# ordenar em ordem decrescente as variáveis por seus valores ausentes

(df.isnull().sum() / df.shape[0]).sort_values(ascending=False)

Copia do DATAFRAME com as variáveis necessárias

In [0]:
# Determinando as colunas que serão copiadas
variaveis = ['CO_UF_RESIDENCIA',	'NU_IDADE',	'TP_SEXO',	'TP_COR_RACA',	'TP_NACIONALIDADE',	'TP_ST_CONCLUSAO',	'TP_ANO_CONCLUIU',	'TP_ESCOLA',	'IN_TREINEIRO',	'TP_PRESENCA_CN',	'TP_PRESENCA_CH',	'TP_PRESENCA_LC',	'NU_NOTA_CN',	'NU_NOTA_CH',	'NU_NOTA_LC',	'NU_NOTA_MT']
df_clean = df[variaveis]

Nossos novos dados

In [0]:
df_clean.head()

Atualiza as notas nulas com ZERO

In [0]:
#Preenche valores nulos com zero para as notas
df_clean.NU_NOTA_MT.fillna(0,inplace=True)
df_clean.NU_NOTA_LC.fillna(0,inplace=True)
df_clean.NU_NOTA_CH.fillna(0,inplace=True)
df_clean.NU_NOTA_CN.fillna(0,inplace=True)

Conversão do sexo para O e 1

In [0]:
# converter 'Sexo' em 0 e 1
df_clean['TP_SEXO'] = df_clean['TP_SEXO'].map({'M': 0, 'F': 1})

Verificando valores nulos

In [0]:
df_clean.isnull().sum()

In [0]:
df_clean.head()

In [0]:
# encontrando as colunas com valores não unicos
df_clean.nunique(axis=0)

In [0]:
# definindo nossas features e nosso alvo

X = df_clean.drop('NU_NOTA_MT', axis=1)
y = df_clean.NU_NOTA_MT

In [0]:
# dividindo o dataset entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [0]:
# instanciar e treinar o modelo 
model_simp = XGBRegressor(n_estimators=100,learning_rate=0.05,gamma=0.1,random_state=0, silent=True)
model_simp.fit(X_train, y_train)


In [0]:
# fazer as previsões em cima do dataset de teste
y_pred = model_simp.predict(X_test)

In [0]:
# métricas de avaliação
print("r2: \t{:.4f}".format(r2_score(y_test, y_pred)))
print("MAE: \t{:.4f}".format(mean_absolute_error(y_test, y_pred)))
print("MSE: \t{:.4f}".format(mean_squared_error(y_test, y_pred)))

Conseguimos 92% de acurácia neste primeiro modelo

Realizando nossa predicao a partir dos dados 

In [0]:
# Importando dados 
df_prev = pd.read_csv('test.csv')

In [0]:
df_prev.shape

In [0]:
# Determinando as colunas que serão copiadas
variaveis = ['CO_UF_RESIDENCIA',	'NU_IDADE',	'TP_SEXO',	'TP_COR_RACA',	'TP_NACIONALIDADE',	'TP_ST_CONCLUSAO',	'TP_ANO_CONCLUIU',	'TP_ESCOLA',	'IN_TREINEIRO',	'TP_PRESENCA_CN',	'TP_PRESENCA_CH',	'TP_PRESENCA_LC',	'NU_NOTA_CN',	'NU_NOTA_CH',	'NU_NOTA_LC']
df_sub = df_prev[variaveis]

In [0]:
# converter 'Sexo' em 0 e 1
df_sub['TP_SEXO'] = df_sub['TP_SEXO'].map({'M': 0, 'F': 1})

In [0]:
# Aplicando o nosso modelo aos dados
y_sub = model_simp.predict(df_sub)

In [0]:
# criando nosso arquivo para submeter para avaliação
sub = pd.Series(y_sub, index = df_prev['NU_INSCRICAO'], name = 'NU_NOTA_MT')

sub.to_csv('answer.csv', header=True)